# Assignment 1 - Part 3
Fine-tuning the BM25 variables to yield the best results

In [46]:
from elasticsearch import Elasticsearch

In [47]:
INDEX_NAME = "aquaint"
DOC_TYPE = "doc"

In [48]:
QUERY_FILE = "data/queries.txt"  # make sure the query file exists on this location

In [49]:
OUTPUT_FILE = "data/bm25_improved.txt"  # output the ranking

### Load the queries from the file

See the assignment description for the format of the query file [here](https://github.com/kbalog/uis-dat630-fall2017/tree/master/assignment-1#queries).

In [50]:
def load_queries(query_file):
    queries = {}
    with open(query_file, "r") as fin:
        for line in fin.readlines():
            qid, query = line.strip().split(" ", 1)
            queries[qid] = query
    return queries

In [51]:
queries = load_queries(QUERY_FILE)

### Perform retrieval

In [52]:
#Default values of k1 & b
#b = 0.75
#k1 = 1.2
es = Elasticsearch()
print(es.indices.get_settings(index=INDEX_NAME))
#Closing index because you cannot change 'dynamic' settings on an open index
es.indices.close(index=INDEX_NAME)
es.indices.put_settings(index=INDEX_NAME, body={
    "similarity": {
        "default": {
            "type": "BM25",
            "b": 0.400,
            "k1": 1.090
        }
    }
})
es.indices.open(index=INDEX_NAME)
print(es.indices.get_settings(index=INDEX_NAME))

{'aquaint': {'settings': {'index': {'number_of_replicas': '1', 'creation_date': '1504824745596', 'similarity': {'default': {'b': '0.4', 'type': 'BM25', 'k1': '1.09'}}, 'uuid': 'QzU6kO-FR66GJpeq6JeBFg', 'provided_name': 'aquaint', 'version': {'created': '5050299'}, 'number_of_shards': '1'}}}}
{'aquaint': {'settings': {'index': {'number_of_replicas': '1', 'creation_date': '1504824745596', 'similarity': {'default': {'b': '0.4', 'type': 'BM25', 'k1': '1.09'}}, 'uuid': 'QzU6kO-FR66GJpeq6JeBFg', 'provided_name': 'aquaint', 'version': {'created': '5050299'}, 'number_of_shards': '1'}}}}


In [53]:

with open(OUTPUT_FILE, 'w') as out:
    out.write("QueryId,DocumentId\n")
    for q_id in sorted(queries):
        print("Ranking documents for [%s] '%s'" % (q_id, queries[q_id]))
        res = es.search(index=INDEX_NAME, q=queries[q_id], _source=False, size=100)
        for item in res['hits']['hits']:
            out.write(q_id + "," + item['_id'] + "\n") 
            


Ranking documents for [303] 'Hubble Telescope Achievements'
Ranking documents for [307] 'New Hydroelectric Projects'
Ranking documents for [310] 'Radio Waves and Brain Cancer'
Ranking documents for [314] 'Marine Vegetation'
Ranking documents for [322] 'International Art Crime'
Ranking documents for [325] 'Cult Lifestyles'
Ranking documents for [330] 'Iran-Iraq Cooperation'
Ranking documents for [336] 'Black Bear Attacks'
Ranking documents for [341] 'Airport Security'
Ranking documents for [344] 'Abuses of E-Mail'
Ranking documents for [345] 'Overseas Tobacco Sales'
Ranking documents for [347] 'Wildlife Extinction'
Ranking documents for [353] 'Antarctica exploration'
Ranking documents for [354] 'journalist risks'
Ranking documents for [362] 'human smuggling'
Ranking documents for [363] 'transportation tunnel disasters'
Ranking documents for [367] 'piracy'
Ranking documents for [372] 'Native American casino'
Ranking documents for [374] 'Nobel prize winners'
Ranking documents for [375] 'h